In [1]:
import pymongo
import pandas as pd
import numpy as np
import datetime

In [2]:
# Stupid but smart
def getCorrespondingStartDate(date):
    month = date.month
    year = date.year
    if month - 10 >= 0: 
        return datetime.date(year, 10, 1)
    if month - 7 >= 0:
        return datetime.date(year, 7, 1)
    if month - 4 >= 0:
        return datetime.date(year, 4, 1)
    return datetime.date(year, 1, 1)

In [3]:
# here's my plan. i iterate through features separately. so i can sort by $exists = True.
# helps avoid those annoying key errors, and the mathematical badness of setting nan = 0
# when you're trying to calculate an average. 
# it's NOT efficient, but we're only doing it once. right??

# shift fields: fields shifted back by -4 (so we look ahead to one year later)
def getDataAnomQuarterly(start_date, end_date, stocks, non_shift_fields, shift_fields=None):
    client = pymongo.MongoClient('localhost', 27017, maxPoolSize=100)
    db = client['Stocks']

    fields = non_shift_fields + shift_fields
    cols = ['Stock'] + fields

    date_index = pd.date_range(getCorrespondingStartDate(start_date),
                               getCorrespondingStartDate(end_date),
                               freq='QS')
    
    df_arr = [None for _ in range(len(stocks))]
    
    curr_date = None
    curr_total = 0
    curr_count = 0
    
    for i in range(len(stocks)): 
        
        stock = stocks[i]
        df = pd.DataFrame([], columns=cols, index=date_index)
        df['Stock'] = stock
        
        for field in fields:
            
            curr_stock = db[stock]
            
            for thing in curr_stock.find({field:{'$exists': True}}).sort([('Date', 1)]):
                date = thing['Date']
                if date > end_date:
                    break
                if date < start_date:
                    continue
                    
                corr_date = getCorrespondingStartDate(date)

                if curr_date is None:
                    curr_count += 1
                    curr_date = corr_date
                    curr_total = thing[field]
                elif (corr_date == curr_date):
                    curr_count += 1
                    curr_total += thing[field]   
                
                # If the dates don't match, we save the info before moving on. 
                else:
                    df.at[curr_date, field] = curr_total/curr_count
                    curr_date = corr_date
                    curr_count = 1
                    curr_total = thing[field]

        
        # Do the shifting
        for sfield in shift_fields:
            shifted = df[sfield].shift(periods=-4)
            new_name = sfield + ' in 1 yr'
            df[new_name] = shifted
        
        df_arr[i] = df
            
    return pd.concat(df_arr).sort_index()


In [4]:
start_date = datetime.datetime(2000, 1, 1)
end_date = datetime.datetime(2005, 1, 1)

stocks = ["AAPL US Equity", "WWW US Equity"]

fields = ['EBITDA']
shift_fields = ['LAST_PRICE']

dataDict = getDataAnomQuarterly(start_date, end_date, stocks, fields, shift_fields)

In [5]:
print(dataDict)

                     Stock   EBITDA LAST_PRICE LAST_PRICE in 1 yr
2000-01-01  AAPL US Equity      NaN    4.21653            1.47487
2000-01-01   WWW US Equity  21.5895     3.5139            4.95553
2000-04-01   WWW US Equity   21.337    3.68057            5.85223
2000-04-01  AAPL US Equity      191    3.72397            1.63547
2000-07-01   WWW US Equity   -8.131     3.4236             5.5722
2000-07-01  AAPL US Equity    147.5    3.27383            1.25833
2000-10-01  AAPL US Equity     -396     1.2128            1.44667
2000-10-01   WWW US Equity   21.872     4.2361            4.97447
2001-01-01  AAPL US Equity       14    1.47487             1.6688
2001-01-01   WWW US Equity   14.905    4.95553            5.37553
2001-04-01  AAPL US Equity       68    1.63547            1.55453
2001-04-01   WWW US Equity   19.331    5.85223            5.90447
2001-07-01  AAPL US Equity       83    1.25833            1.05977
2001-07-01   WWW US Equity   27.946     5.5722               4.98
2001-10-01